In [ ]:
!pip install Flask-SQLAlchemy
!pip install Flask-login
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy 
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SECRET_KEY'] = 'thisissecret'


db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "重新登陆"
login_manager.refresh_view = '登陆'
login_manager.login_view = '登陆'
    
class User(UserMixin, db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50),unique=True)
    email = db.Column(db.String(255),unique=True)
    password = db.Column(db.String(80))
    def __repr__(self):
        return '<User %r>' % self.id


@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route("/")
def home():
    status = "home"
    return render_template('index.html', status=status)


@app.route('/login', methods=["GET", "POST"])
def login():
    message=''
    status = "login"
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        print(username,password)

        user = User.query.filter_by(username=username).first()
        if not user:
            print("用户名错误或未注册")
            flash("用户名错误或未注册")#for website
            return redirect(url_for('login'))
        else:
            if password == user.password:
                print("登陆成功")
                login_user(user)
                flash("登陆成功")
                return redirect(url_for('profile'))
            else:
                print("密码错误")
                flash("密码错误,再想想?")
                return redirect(url_for('login'))           
    
    else:
        return render_template('login.html',status=status)

@app.route("/profile")
@login_required
def profile():
    status = "profile"
    return render_template('profile.html',username=current_user.username, status=status)


@app.route("/signup", methods=["GET", "POST"])
def signup():
    status = "signup"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = User(username=username,email=email,password=password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("注册成功!")
            return redirect(url_for('home'))
        except Exception as e:
            print("注册失败",e)
            return '啊偶?注册失败'
    else:
        return render_template('signup.html',status=status)

@app.route("/logout")
@login_required
def logout():
    print('用户登出')
    flash('已登出', current_user.username)
    logout_user()
    return redirect(url_for('home'))


@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == "__main__":
    app.run('localhost', 9408)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/bian/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9408/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2021 18:13:29] "GET / HTTP/1.1" 200 -
